In [1]:
!pip install flask

In [6]:
import flask
import json
from functools import wraps
from flask import request, Response
import requests
from bs4 import BeautifulSoup
import urllib.request
from powernad.API import RelKwdStat
import time
from random import uniform
import os

In [8]:
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [9]:
#네이버 api 
Client_id = os.getenv("NAVER_CLIENT_ID")
Client_Secret = os.getenv("NAVER_CLIENT_SECRET")

#네이버 광고 api
NAVER_AD_CUSTOMER_ID = os.getenv("NAVER_AD_CUSTOMER_ID")
NAVER_AD_ACCESS_LICENSE =os.getenv("NAVER_AD_API_KEY")
NAVER_AD_SECRET_KEY = os.getenv("NAVER_AD_SECRET_KEY")

#네이버 블로그 api 요청 주소
NAVER_BLOG_API_URL="https://openapi.naver.com/v1/search/blog?query="
#네이버 쇼핑 api 요청 주소
NAVER_SHOP_API_URL="https://openapi.naver.com/v1/search/shop?query="
#네이버 광고 api 요청 주소
NAVER_AD_API_URL = "https://api.naver.com"

In [ ]:
# 초당 몇회 이상일 경우 접근제한이 결려있기에 조절
LIMIT=10

app= flask.Flask(__name__)

#리턴값 변환 함수
def as_json(f):
    @wraps(f)
    def decorated_function(*args, **kwargs):
        res = f(*args, **kwargs)
        res = json.dumps(res, ensure_ascii=False).encode('utf8')
        return Response(res, content_type='application/json; charset=utf-8')

    return decorated_function


#네이버 api 카운트 가져오기 함수
def getSearchCount(keyword,URL):
    encText = urllib.parse.quote(keyword) 
    url=URL + encText
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",Client_id)
    request.add_header("X-Naver-Client-Secret",Client_Secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    response_body = response.read()
    jsonString = response_body.decode("utf-8")
    jsonDict =json.loads(jsonString)
    items = jsonDict['items']

    for item in items:
        title = item['title']
        link =item['link']
    if(rescode==200):
        totalCount = jsonDict["total"]
    else:
        totalCount = 0
    return totalCount

#naver api 가져오기 함수
def getSearchList(keyword,URL):
    searchList =[]
    encText = urllib.parse.quote(keyword) 
    url=URL + encText
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",Client_id)
    request.add_header("X-Naver-Client-Secret",Client_Secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    response_body = response.read()
    jsonString = response_body.decode("utf-8")
    jsonDict =json.loads(jsonString)
    items = jsonDict['items']

    for item in items:
        title = item['title']
        link =item['link']
        searchList.append({'title':title,'link': link})
        print(searchList)
    
    return searchList

# naver api 블로그 글 가져오기
@app.route('/getBlogs', methods=['GET'])
@as_json
def getBlogs():
    if 'keyword' in request.args:
        keyword = str(request.args['keyword'])
    else:
        return "Error: keyword field was not provided. pleasw enter a keyword."
    
    return getSearchList(keyword,NAVER_BLOG_API_URL)

#naver api 쇼핑정보 가져오기
@app.route('/getShops', methods=['GET'])
@as_json
def getShops():
    if 'keyword' in request.args:
        keyword = str(request.args['keyword'])
    else:
        return "Error: keyword field was not provided. pleasw enter a keyword."
    
    return getSearchList(keyword,NAVER_SHOP_API_URL)
    

#naver api 탑10 가져오기
@app.route('/getPopularlists', methods=['GET'])
@as_json
def popularlist():
    NAVER_BEST100 = 'https://search.shopping.naver.com/best100v2/main.nhn'
    popular10lists =[]

    source = requests.get(NAVER_BEST100).text
    soup = BeautifulSoup(source,"html.parser")
    popular10 = soup.find(id="popular_srch_lst")
    popular10names = popular10.select(".txt")
    for name in popular10names:
        popular10lists.append({"name" : name.text})
        
    return popular10lists


#naver api  연관검색어,클릭 횟수 등 가져오기
@app.route('/relatedkeywords',methods=['GET'])
@as_json
def relatedkeywords():
    keywords =[]
    if 'keyword' in request.args:
        keyword = str(request.args['keyword'])
    else:
        return "Error: keyword field was not provided. pleasw enter a keyword."

    relKwdStat = RelKwdStat.RelKwdStat(NAVER_AD_API_URL, NAVER_AD_ACCESS_LICENSE,NAVER_AD_SECRET_KEY, NAVER_AD_CUSTOMER_ID)
    KWDataList = relKwdStat.get_rel_kwd_stat_list(None,hintKeywords=keyword,showDetail='1')
    for idx, outdata in enumerate(KWDataList):
        time.sleep(uniform(0.11,0.12)) #naver search api limit - daily 2500 max 10/sec
        relKeyword = outdata.relKeyword
        monthlyPcQcCnt = outdata.monthlyPcQcCnt
        monthlyMobileQcCnt = outdata.monthlyMobileQcCnt
        monthlyAvePcCtr = outdata.monthlyAvePcCtr
        monthlyAveMobileCtr = outdata.monthlyAveMobileCtr

        compIdx = outdata.compIdx

        blogsTotal = getSearchCount(relKeyword, NAVER_BLOG_API_URL)
        shopsTotal = getSearchCount(relKeyword, NAVER_SHOP_API_URL)

        if(str(monthlyPcQcCnt).isnumeric() and str(monthlyMobileQcCnt).isnumeric() and compIdx =="높음"):
            totalCnt =monthlyPcQcCnt + monthlyMobileQcCnt
            clickCnt = round(monthlyAvePcCtr + monthlyAveMobileCtr ,1)
            print(idx, relKeyword, totalCnt, clickCnt, blogsTotal, shopsTotal)
            keywords.append({'word':relKeyword, 'totalCnt':totalCnt,'clickCnt':clickCnt,'blogsTotal':blogsTotal})
            if(idx>=LIMIT):
                break
            
    return keywords

app.run()